# Week 4.
**Deliverables**
* Refine prompts for Binary & Multitype

## 1. Prompt Library
Binary and multitype templates that feed every evaluation routine.


In [ ]:
# BINARY PROMPTS
# (1) Zero-Shot — final (no inline JSON examples)
prompt_binary_zeroshot = """
You are analyzing whether a sentence describes Intimate Partner Violence (IPV).

Definition:
Intimate Partner Violence includes any form of physical, emotional, or sexual harm
inflicted by a partner. This may involve control, manipulation, threats, humiliation,
or physical aggression.

Decide if the following sentence involves IPV or not.

Sentence: "{text}"

Return ONLY one JSON object, enclosed between <json> and </json>.
Format: a single key "label" with value "IPV" or "NOT_IPV".
Print nothing else.
""".strip()


# (2) Few-Shot — final (no inline JSON examples)
prompt_binary_fewshot = """
You are classifying sentences as describing Intimate Partner Violence (IPV) or NOT_IPV.

Definition:
IPV occurs when one partner harms or controls another physically, emotionally, or sexually.

Here are examples (for guidance only):
1. "He pushed me against the wall and shouted that I’m worthless." → IPV
2. "She often insults me and forbids me from seeing my friends." → IPV
3. "We argued but both apologized and moved on peacefully." → NOT_IPV
4. "He surprised me with flowers after work." → NOT_IPV

Now analyze this sentence:
"{text}"

Return ONLY one JSON object, enclosed between <json> and </json>.
Format: {"label": "IPV"} or {"label": "NOT_IPV"}.
Print nothing else.
""".strip()


# (3) Chain-of-Thought — final (no inline JSON examples)
prompt_binary_cot = """
You are reasoning step-by-step to decide if a sentence describes Intimate Partner Violence (IPV).

Guidelines:
- IPV includes threats, coercion, physical harm, humiliation, or emotional manipulation.
- NOT_IPV describes healthy, neutral, or unrelated situations.

Think internally (do NOT show your reasoning) about:
1. Does the sentence show any behavior that causes harm, fear, or control?
2. Is there a partner/relationship context?
3. Does it express affection or support instead of harm?

After thinking silently, return ONLY one JSON object, enclosed between <json> and </json>.
Format: {"label": "IPV"} or {"label": "NOT_IPV"}.
Print nothing else.

Sentence: "{text}"
""".strip()


# (4) Meta — final (no inline JSON examples)
prompt_binary_meta = """
You are a social-behavioral analyst evaluating sentences for signs of Intimate Partner Violence (IPV).

Your objective is to be accurate but cautious.
- If the sentence clearly involves harm, coercion, or control → label as IPV.
- If the sentence shows affection, neutrality, or uncertainty → label as NOT_IPV.

Reflect internally before answering; do NOT print your reasoning.

Sentence: "{text}"

Return ONLY one JSON object, enclosed between <json> and </json>.
Format: {"label": "IPV"} or {"label": "NOT_IPV"}.
Print nothing else.
""".strip()


# (5) Self-Consistency — final (no inline JSON examples)
prompt_binary_selfconsistency = """
You will internally evaluate the sentence for Intimate Partner Violence (IPV) multiple times
and choose the majority label as your final answer.

Guidelines:
- IPV → signs of physical, emotional, or sexual harm, threats, or coercion.
- NOT_IPV → supportive, neutral, or unrelated content.

Do NOT reveal thoughts or votes.

Sentence: "{text}"

Return ONLY one JSON object, enclosed between <json> and </json>.
Format: {"label": "IPV"} or {"label": "NOT_IPV"}.
Print nothing else.
""".strip()

### 1.2 Multitype Prompts
These templates mirror the binary ones but collect subtype labels.


In [ ]:
# MULTITYPE PROMPTS
prompt_multilabel_zeroshot = """
You are identifying which forms of Intimate Partner Violence (IPV) appear in a sentence.

Decide independently for emotional, physical, and sexual abuse.
Return ONLY one JSON object enclosed between <json> and </json> with the keys
'id', 'emotional', 'physical', and 'sexual'.

Sentence: "{text}"
Sample ID: "{sample_id}"
<json>
{{
  "id": "{sample_id}",
  "emotional": 0 or 1,
  "physical": 0 or 1,
  "sexual": 0 or 1
}}
</json>
""".strip()

prompt_multilabel_fewshot = """
You are labeling sentences for types of Intimate Partner Violence (IPV).
Use the examples to stay calibrated. For each category (emotional, physical, sexual), output 1 if it is clearly present, else 0.

Examples:
1. "He insults me daily and forbids me from leaving the house." -> emotional: 1, physical: 0, sexual: 0
2. "She slapped me when I disagreed with her." -> emotional: 0, physical: 1, sexual: 0
3. "They pressured me into intimacy when I said no." -> emotional: 0, physical: 0, sexual: 1
4. "We spent the evening cooking together peacefully." -> emotional: 0, physical: 0, sexual: 0

Sentence: "{text}"
Sample ID: "{sample_id}"
<json>
{{
  "id": "{sample_id}",
  "emotional": 0 or 1,
  "physical": 0 or 1,
  "sexual": 0 or 1
}}
</json>
""".strip()

prompt_multilabel_cot = """
Reason silently about whether emotional, physical, or sexual IPV occurs in the sentence.
Use relationship context, threats, coercion, and bodily harm cues.
After your hidden reasoning, output ONLY the JSON block specified below.

Sentence: "{text}"
Sample ID: "{sample_id}"
<json>
{{
  "id": "{sample_id}",
  "emotional": 0 or 1,
  "physical": 0 or 1,
  "sexual": 0 or 1
}}
</json>
""".strip()

prompt_multilabel_meta = """
Act as a cautious social-behavioral analyst.
Label a subtype as 1 only when the text clearly shows that form of IPV; otherwise return 0.
Favor precision to avoid false positives.

Sentence: "{text}"
Sample ID: "{sample_id}"
<json>
{{
  "id": "{sample_id}",
  "emotional": 0 or 1,
  "physical": 0 or 1,
  "sexual": 0 or 1
}}
</json>
""".strip()

prompt_multilabel_selfconsistency = """
Independently evaluate the sentence multiple times to reduce uncertainty.
After internal self-consistency voting, output the majority decision for each subtype in the JSON schema below.
Do not reveal the intermediate thoughts.

Sentence: "{text}"
Sample ID: "{sample_id}"
<json>
{{
  "id": "{sample_id}",
  "emotional": 0 or 1,
  "physical": 0 or 1,
  "sexual": 0 or 1
}}
</json>
""".strip()


## 2. System & Model Setup
Shared imports, paths, and model objects. Execute once per runtime.


In [ ]:
from __future__ import annotations

import json
import math
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
DATASET_PATH = Path("../Dataset/617points.csv")
RESULTS_ROOT = Path("test_results_raw")
BINARY_RESULTS_DIR = RESULTS_ROOT / "binary"
MULTITYPE_RESULTS_DIR = RESULTS_ROOT / "multitype"

print(f"Loading {MODEL_NAME} ...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
model.eval()
print("Model ready.")


## 3. Prediction Generation
Run the Colab cloning cell first, then load the dataset and choose either binary or multitype generation.


In [ ]:
#Clone from git
!git clone https://github.com/zelaneroz/ipvresearch25
%cd ipvresearch25/1_LLM_Eval
#Load dataset
filename = "../Dataset/617points.csv"
df = pd.read_csv(filename)

### 3.0 Data Access & Directories
Loads the dataset and prepares local folders for saving model outputs.


In [ ]:
# Dataset + result folders
BINARY_RESULTS_DIR.mkdir(parents=True, exist_ok=True)
MULTITYPE_RESULTS_DIR.mkdir(parents=True, exist_ok=True)

if not DATASET_PATH.exists():
    raise FileNotFoundError(f"Dataset not found at {DATASET_PATH}. Run the git clone cell above or update DATASET_PATH.")

df = pd.read_csv(DATASET_PATH)
print(f"Loaded {len(df)} rows from {DATASET_PATH}")
print(df.head(2))


In [ ]:
# Prompt registries for downstream loops
binary_prompts = {
    "zeroshot": prompt_binary_zeroshot,
    "fewshot": prompt_binary_fewshot,
    "cot": prompt_binary_cot,
    "meta": prompt_binary_meta,
    "selfconsistency": prompt_binary_selfconsistency,
}

multilabel_prompts = {
    "zeroshot": prompt_multilabel_zeroshot,
    "fewshot": prompt_multilabel_fewshot,
    "cot": prompt_multilabel_cot,
    "meta": prompt_multilabel_meta,
    "selfconsistency": prompt_multilabel_selfconsistency,
}


### 3.1 Binary Prediction Generation

In [ ]:
def test_binary_prompts(df: pd.DataFrame, path: Path, n_samples: int = 3) -> None:
    """Run all binary prompt types on the first `n_samples` rows and persist outputs."""
    import re

    df_subset = df.head(n_samples)
    results_dir = Path(path)
    results_dir.mkdir(parents=True, exist_ok=True)

    print("Running binary classification tests...")
    print(f"Number of samples: {len(df_subset)}")
    print(f"Results will be saved in: {results_dir}
")

    for prompt_type, template in binary_prompts.items():
        print(f"Testing prompt type: {prompt_type}")
        records = []

        for i, row in df_subset.iterrows():
            text = row["items"] if "items" in df.columns else str(row.iloc[0])
            prompt_text = template.replace("{text}", text)

            try:
                inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
                output = model.generate(
                    **inputs,
                    max_new_tokens=128,
                    temperature=0.0,
                    do_sample=False,
                )
                gen_tokens = output[0][inputs["input_ids"].shape[-1]:]
                result_text = tokenizer.decode(gen_tokens, skip_special_tokens=True)
            except Exception as exc:
                result_text = f"ERROR: {exc}"

            label = None
            match = re.search(r"<json[^>]*>\s*(.*?)\s*</json>", result_text, re.DOTALL | re.IGNORECASE)
            if match:
                block = match.group(1).strip()
                try:
                    parsed = json.loads(block)
                    if isinstance(parsed, dict):
                        label = parsed.get("label") or parsed.get("labels")
                    elif isinstance(parsed, list) and parsed:
                        label = parsed[0]
                    elif isinstance(parsed, str):
                        label = parsed.strip()
                except json.JSONDecodeError:
                    if "NOT_IPV" in block.upper():
                        label = "NOT_IPV"
                    elif "IPV" in block.upper():
                        label = "IPV"
            else:
                if "NOT_IPV" in result_text.upper():
                    label = "NOT_IPV"
                elif "IPV" in result_text.upper():
                    label = "IPV"

            if label is None:
                label = "UNKNOWN"

            records.append(
                {
                    "id": int(i),
                    "prompt_type": prompt_type,
                    "extracted_label": label,
                    "raw_response": result_text,
                }
            )

        output_path = results_dir / f"binary_{prompt_type}.json"
        with open(output_path, "w", encoding="utf-8") as fp:
            json.dump(records, fp, indent=4, ensure_ascii=False)

        print(f"Saved results for '{prompt_type}' to {output_path}")

    print("
All binary prompt tests completed.")


In [ ]:
# Binary generation runner (toggle RUN_BINARY to execute)
RUN_BINARY = False
BINARY_SAMPLE_COUNT = 5
binary_run_dir = BINARY_RESULTS_DIR / datetime.now().strftime("%Y%m%d-%H%M%S")

if RUN_BINARY:
    if 'df' not in globals():
        raise RuntimeError("Dataset `df` not loaded. Run the data prep cell above.")
    test_binary_prompts(df, path=binary_run_dir, n_samples=BINARY_SAMPLE_COUNT)
else:
    print("Binary generation skipped. Set RUN_BINARY = True to execute.")


### 3.2 Multitype Prediction Generation

In [ ]:
# ---------- Stage 1: Classification ----------
def multitype_predict(sentence: str, sample_id: Optional[int] = None, prompt_key: str = "zeroshot") -> Dict[str, int]:
    template = multilabel_prompts.get(prompt_key, prompt_multilabel_zeroshot)
    prompt = template.format(text=sentence, sample_id=sample_id or 0)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=False,
    )
    decoded = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    try:
        return json.loads(decoded)
    except Exception:
        import re
        match = re.search(r"\{.*\}", decoded, re.DOTALL)
        return json.loads(match.group()) if match else {"emotional": 0, "physical": 0, "sexual": 0}


# ---------- Stage 2: Confidence ----------
def logprob_confidence(prompt: str, generated_text: str) -> float:
    tokens = tokenizer(prompt + generated_text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**tokens)
        log_probs = torch.nn.functional.log_softmax(outputs.logits, dim=-1)
    input_ids = tokens["input_ids"][0]

    logp_sum = 0.0
    count = 0
    for idx in range(1, len(input_ids)):
        token_id = input_ids[idx]
        logp_sum += log_probs[0, idx - 1, token_id].item()
        count += 1
    avg_logp = logp_sum / max(1, count)
    confidence = math.exp(avg_logp)
    return float(max(0.0, min(1.0, confidence)))


# ---------- Combined Function ----------
def multitype_classify(sentence: str, sample_id: Optional[int] = None, prompt_key: str = "zeroshot") -> Dict[str, float]:
    pred = multitype_predict(sentence, sample_id=sample_id, prompt_key=prompt_key)
    classification_prompt = multilabel_prompts.get(prompt_key, prompt_multilabel_zeroshot).format(
        text=sentence,
        sample_id=sample_id or 0,
    )
    output_str = json.dumps(pred)
    conf = logprob_confidence(classification_prompt, output_str)
    pred["confidence_score"] = round(conf, 4)
    pred["id"] = sample_id or 0
    return pred


In [ ]:
# Multitype generation runner (toggle RUN_MULTITYPE to execute)
RUN_MULTITYPE = False
MULTITYPE_SAMPLE_COUNT = 5
MULTITYPE_PROMPT_KEY = "zeroshot"
multitype_outputs = []

if RUN_MULTITYPE:
    if 'df' not in globals():
        raise RuntimeError("Dataset `df` not loaded. Run the data prep cell above.")
    subset = df.head(MULTITYPE_SAMPLE_COUNT)
    for idx, row in subset.iterrows():
        text = row["items"] if "items" in df.columns else str(row.iloc[0])
        multitype_outputs.append(
            multitype_classify(text, sample_id=int(idx), prompt_key=MULTITYPE_PROMPT_KEY)
        )
    print(f"Generated {len(multitype_outputs)} multitype predictions using '{MULTITYPE_PROMPT_KEY}'.")
else:
    print("Multitype generation skipped. Set RUN_MULTITPE = True to execute.")


## 4. Results
Summaries, metrics, and visual diagnostics. Load the JSON artifacts generated above and feed them into the eval pipeline.


In [ ]:
# Utility: list recently generated result files
from glob import glob

def list_result_files(root: Path, pattern: str = "*.json", limit: int = 10):
    files = sorted(root.rglob(pattern))[-limit:]
    if not files:
        print(f"No files found under {root}")
        return
    for file in files:
        print(f"- {file.relative_to(Path.cwd())}")

print("Recent binary result files:")
list_result_files(BINARY_RESULTS_DIR)
print("
Recent multitype result files:")
list_result_files(MULTITYPE_RESULTS_DIR)
